<center><h3>Problem :- Opening A Restaurant in Chicago City. </h3></center>

<b>Preparing data for the Business Problem.</b>




In [1]:
##Import to read dataframe
#import types
#import pandas as pd
#from botocore.client import Config
#import ibm_boto3

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Load the crime data into a dataframe

In [4]:
crime_data = pd.read_csv('Chicago_Crime_Data-v2.csv')
Chicago_census_data = pd.read_csv('Chicago_socio_economic.csv')
print('data loaded')

data loaded


In [5]:
crime_data.head()


,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,UPDATEDON,LATITUDE,LONGITUDE,LOCATION
0,3512276,HK587712,08/28/2004 05:50:56 PM,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,False,False,911,9,14.0,58.0,6,1155838.0,1873050.0,2004,02/10/2018 03:50:01 PM,41.807441,-87.703956,"(41.8074405, -87.703955849)"
1,3406613,HK456306,06/26/2004 12:40:00 PM,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,False,False,1112,11,27.0,23.0,6,1152206.0,1906127.0,2004,02/28/2018 03:56:25 PM,41.898280,-87.716406,"(41.898279962, -87.716405505)"
2,8002131,HT233595,04/04/2011 05:45:00 AM,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,False,False,221,2,3.0,38.0,6,1177436.0,1876313.0,2011,02/10/2018 03:50:01 PM,41.815933,-87.624642,"(41.815933131, -87.624642127)"
3,7903289,HT133522,12/30/2010 04:30:00 PM,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,423,4,7.0,46.0,6,1194622.0,1850125.0,2010,02/10/2018 03:50:01 PM,41.743665,-87.562463,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,02/02/2016 07:30:00 PM,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,False,False,831,8,15.0,66.0,6,1155240.0,1860661.0,2016,02/10/2018 03:50:01 PM,41.773455,-87.706480,"(41.773455295, -87.706480471)"


In [9]:
# create all crime incidents by ward number 
crime_incidents = crime_data.groupby(['WARD']).size().reset_index(name="Count")
crime_incidents.shape

(50, 2)

In [31]:
crime_incidents.head()
#crime_incidents['WARD'] = crime_incidents['WARD'].astype(str)
crime_incidents.dtypes

WARD     object
Count     int64
dtype: object

Let's get the geographical location of chicago.

In [11]:
address = 'Chicago, Illinois, USA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of chicago are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of chicago are 41.8755616, -87.6244212.


Lets get chicago ward boundaries

In [12]:
Chicago_geo = r'Boundaries - Wards (2015-).geojson' # geojson file
print('data loaded')

data loaded


Creating a folim map to show chicago wards with number of crime incidents

In [32]:
# create a plain world map
world_chicago_map = folium.Map(location=[latitude, longitude], zoom_start=10)
world_chicago_map.choropleth(
    geo_data = Chicago_geo,
    data = crime_incidents,
    columns = ['WARD', 'Count'],
    key_on = 'feature.properties.WARD',    
    #threshold_scale = threshold_scale,
    fill_color = 'YlOrRd', 
    fill_opacity = 0.7, 
    line_opacity = 0.2,
    legend_name = 'Crime Rates in Chicago',  
    #threshold_scale = [0 , 5, 10]
)
world_chicago_map